https://www.kaggle.com/code/odins0n/spaceship-titanic-eda-27-different-models : 0.79494を参考

In [ ]:
from IPython.display import clear_output
!pip3 install -U lazypredict
!pip3 install -U pandas #Upgrading pandas

clear_output()

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots



from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split


from lightgbm import LGBMClassifier
import lazypredict
from lazypredict.Supervised import LazyClassifier

import time
import warnings
warnings.filterwarnings('ignore')


In [ ]:
train = pd.read_csv("../input/spaceship-titanic/train.csv")
test = pd.read_csv("../input/spaceship-titanic/test.csv")
submission = pd.read_csv("../input/spaceship-titanic/sample_submission.csv")

RANDOM_STATE = 12 
FOLDS = 5
STRATEGY = 'median'

In [ ]:
train

In [ ]:
print(f'Number of rows in train data: {train.shape[0]}')
print(f'Number of columns in train data: {train.shape[1]}')
print(f'Number of values in train data: {train.count().sum()}')
print(f'Number missing values in train data: {sum(train.isna().sum())}')

In [ ]:
TARGET = 'Transported'
# train.drop(["Name" ,"Cabin","PassengerId"] , axis = 1 ,inplace = True)
# test.drop(["Name" ,"Cabin"] , axis = 1 ,inplace = True)

In [ ]:
def preprocess(df):
    df["HomePlanet"] = df["HomePlanet"].fillna('Z')
    # Cabinカラム 分割＋生成
    df = pd.concat([df, df['Cabin'].str.split('/', expand = True).rename({0:'deck',1:'num',2:'side'},axis=1)],axis=1)
    df = pd.concat([df, df['PassengerId'].str.split('_', expand = True).rename({0:'group_id',1:'passenger_id'},axis=1)],axis=1)
    df['num'] = df['num'].fillna(10000)
    df['num'] = df['num'].astype('int')
    df['group_id'] = df['group_id'].astype('int')
    df['passenger_id'] = df['passenger_id'].astype('int')

    # 追加特徴量
    df['ages'] = df['Age'].apply(lambda x : x%10) #年代 10歳ごと
    df = df.merge(df.groupby('group_id')['num'].count().reset_index().rename({'num':'group_count'},axis=1), how = 'left', on = 'group_id') #グループの人数特徴量
     #log特徴量 (あまり効果なし)
#     for col in ['RoomService','FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']:
#         df[f'log_{col}'] = np.log(df[col])
    df['vr_spend']=df['RoomService']+df['FoodCourt']+df['ShoppingMall']+df['Spa']+df['VRDeck']
    df['no_spend'] = df['vr_spend'].apply(lambda row: 1 if row == 0 else 0)
    df['LastName'] = df.Name.str.split(' ').str[1]
    df['relative_group'] = df.groupby(['group_id'])['LastName'].transform('count')
    df['isChild'] = df['Age'].apply(lambda x: 1 if x < 10 else 0)
    df['isOld'] = df['Age'].apply(lambda x: 1 if x > 60 else 0)
    
    df = df.drop(["Name" ,"PassengerId", "Cabin"] , axis = 1 )

    # Label Encoding
    columns = ["HomePlanet", "CryoSleep", "Destination" ,"VIP", 'deck','side','LastName']
    for col in columns:
        df[col] = df[col].astype(str)
        df[col] = LabelEncoder().fit_transform(df[col])
    
    return df

In [ ]:
train = preprocess(train)
test = preprocess(test)

In [ ]:
train.columns

In [ ]:
train.groupby('group_id')['num'].count().reset_index()

In [ ]:
# train['group_count'] = train.groupby('group_id').count()
# train = train.merge(train.groupby('group_id')['num'].count().reset_index().rename({'num':'group_count'},axis=1), how = 'left', on = 'group_id')

In [ ]:
X = train.drop(TARGET , axis =1 )
y = train[TARGET]
X_train , X_test , y_train , y_test = train_test_split(X , 
                                                       y, 
                                                       random_state = 12 ,
                                                       test_size =0.33)

In [ ]:
l = list(train.columns)
l.remove('Transported')
print(l)

In [ ]:
lgb_params = {
    'objective' : 'binary',
    'n_estimators' :50,
    'learning_rate' : 0.08
}

lgb_predictions = 0
lgb_scores = []
lgb_fimp = []
LGBM_FEATURES = list(train.columns)
LGBM_FEATURES.remove('Transported')
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=RANDOM_STATE)
for fold, (train_idx, valid_idx) in enumerate(skf.split(train[LGBM_FEATURES], train[TARGET])):
    print(f'\033[94m')
    print(10*"=", f"Fold={fold+1}", 10*"=")
    start_time = time.time()
    
    X_train, X_valid = train.iloc[train_idx][LGBM_FEATURES], train.iloc[valid_idx][LGBM_FEATURES]
    y_train , y_valid = train[TARGET].iloc[train_idx] , train[TARGET].iloc[valid_idx]
    
    model = LGBMClassifier(**lgb_params)
    model.fit(X_train, y_train,verbose=0)
    
    preds_valid = model.predict(X_valid)
    acc = accuracy_score(y_valid,  preds_valid)
    lgb_scores.append(acc)
    run_time = time.time() - start_time
    
    fim = pd.DataFrame(index=LGBM_FEATURES,
                 data=model.feature_importances_,
                 columns=[f'{fold}_importance'])
    lgb_fimp.append(fim)
    
    print(f"Fold={fold+1}, Accuracy score: {acc:.2f}%, Run Time: {run_time:.2f}s")
    test_preds = model.predict(test[LGBM_FEATURES]) 
    lgb_predictions += test_preds/FOLDS
print("")
print("Mean Accuracy :", np.mean(lgb_scores))

In [ ]:
lgbm_fis_df = pd.concat(lgb_fimp, axis=1)
lgbm_fis_df.sort_values('1_importance').plot(kind='barh', figsize=(15, 10),
                                       title='Feature Importance Across Folds')
plt.show()

In [ ]:
lgbm_fis_df

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in the following fields: PassengerId, HomePlanet, CryoSleep, Destination, VIP

In [ ]:
submission[TARGET] = lgb_predictions.astype("bool")
submission.to_csv("submission.csv",index=False)
submission.head()